In [14]:
# duh
import numpy as np
import pandas as pd
# plot
import matplotlib.pyplot as plt
import matplotlib as mpl
# gibberish
import re 
# scikit
from sklearn import svm
from sklearn.model_selection import train_test_split

# get data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# combine it
data = [train, test]

train.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


In [15]:
# clean up time
for dataset in data:
    # math
    avg = dataset['Age'].mean()
    std = dataset['Age'].std()
    null_count = dataset['Age'].isnull().sum()
    
    # generate and replace missing ages with rdm ones based on stats
    random_ages = np.random.randint(avg - std, avg + std, size=null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = random_ages
    dataset['Age'] = dataset['Age'].astype(int)

# does being older make you more likely to die? ...yes
train['AgeImpact'] = pd.cut(train['Age'], 5)
age_survival = train[['AgeImpact', 'Survived']].groupby(['AgeImpact'], as_index=False).mean()

# do it again for embarking
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
embarked_survival = train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

In [16]:
# $$$
train['FareImpact'] = pd.qcut(train['Fare'], 4)
fare_survival = train[['FareImpact', 'Survived']].groupby(['FareImpact'], as_index=False).mean()

# am I a loner
for dataset in data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
for dataset in data:
    dataset['Loner'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'Loner'] = 1
alone_survival = train[['Loner', 'Survived']].groupby(['Loner'], as_index=False).mean()

# am I important 
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ''

for dataset in data:
    dataset['Title'] = dataset['Name'].apply(get_title)

# how important exactly?
for dataset in data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Jonkheer',
                                                'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Dona'],)
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
title_survival = train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [17]:
for dataset in data:
    dataset['Sex'] = dataset['Sex'].map({'female':0, 'male':1})

    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].median())
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

    title_mapping = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':3, 'Mythical':5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4

    dataset['Embarked'] = dataset['Embarked'].map({'S':0, 'C':1, 'Q':2})
    
# drop undesireables #1-7
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch', 'FamilySize', 'FareImpact', 'AgeImpact'], axis=1)
test_no_id = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch', 'FamilySize'], axis=1) 

train.head()
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Loner,Title
0,892,3,"Kelly, Mr. James",1,2,0,0,330911,0,NaN,2,1,1,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,2,1,0,363272,0,NaN,0,2,0,3
2,894,2,"Myles, Mr. Thomas Francis",1,3,0,0,240276,1,NaN,2,1,1,1
3,895,3,"Wirz, Mr. Albert",1,1,0,0,315154,1,NaN,0,1,1,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,1,1,1,3101298,1,NaN,0,3,0,3


In [18]:
# X contains features, Y contains targets
X = train.iloc[:, 1:]
Y = train.iloc[:, 0]

# banana split the training data for cross validation
X_train, X_cv, Y_train, Y_cv = train_test_split(X, Y, test_size=0.25)

# search the grid
def grid_search():
    C_values = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]
    gamma_values = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]
    kernels = ['linear', 'rbf', 'sigmoid']

    best_score = 0
    best_C = None
    best_gamma = None
    best_kernel = None
    
    for C in C_values:
        for gamma in gamma_values:
            for kernel in kernels:
                svc = svm.SVC(C=C, gamma=gamma, kernel=kernel)
                svc.fit(X_train, Y_train)
                score = svc.score(X_cv, Y_cv)
                
                if score > best_score:
                    best_score = score
                    best_C = C
                    best_gamma = gamma
                    best_kernel = kernel
                    
    print('Best parameters give {0:.4%} accuracy'.format(best_score))
    print('C = {0}\ngamma = {1}\nKernel = {2}'.format(best_C, best_gamma, best_kernel))
    
    return best_C, best_gamma, best_kernel

In [19]:
C, gamma, kernel = grid_search()

Best parameters give 82.5112% accuracy
C = 0.3
gamma = 1
Kernel = rbf


In [20]:
# fitting SVM into a sea of rough boring firey gamma radiation water with 100% probability
svc = svm.SVC(C=3, gamma=0.1, kernel='rbf', probability=True)
svc.fit(X, Y)

# 8ball will my code work?
prediction = svc.predict(test_no_id)[:, np.newaxis]
prediction = np.hstack((test['PassengerId'][:, np.newaxis], prediction))

# it is decidely so
output = pd.DataFrame({'PassengerId':prediction[:, 0], 'Survived':prediction[:, 1]})

# np.savetxt('submission.csv', output, header='PassengerId,Survived', comments='', delimiter=',', fmt='%d')

output.to_csv('hopefully_this_works_titanic_svm.csv', index=False)

output.head()

ValueError: Wrong number of items passed 2, placement implies 1